# Day 13: Transparent Origami
https://adventofcode.com/2021/day/13

### Part 1
First fold.

In [1]:
import os
import re
from typing import Tuple, Optional
from dataclasses import dataclass, field

In [58]:
@dataclass(frozen=True, order=True, eq=True)
class Fold:
    axis:str = field(default='y')
    position: int = field(default=0)

@dataclass(frozen=True, order=True, eq=True)
class Dot:
    x:int
    y:int

class Paper:
    sheet: list[list[Optional[Dot]]]
    folds: list[Fold]
    length:int
    height:int

    def __init__(self, input_file:str):
        if not os.path.exists(input_file):
            raise ValueError(f"input_file {input_file} is not valid.")
        
        dots: list[Dot] = []
        self.folds = []

        with open(input_file) as f:    
            for line in f:
                if line.rstrip():
                    m_fold = re.search('fold along ([x|y]=[0-9]+)', line, re.IGNORECASE)
                    if m_fold:
                        axis, position = m_fold.group(1).split("=")
                        fold = Fold(axis, int(position))
                        self.folds.append(fold)

                    m_dot = re.search('^([0-9]+,[0-9]+)$', line, re.IGNORECASE)
                    if m_dot:
                        x, y = m_dot.group(1).split(",")
                        dot = Dot(int(x), int(y))
                        dots.append(dot)

        if len(dots):
            self.height = max([int(d.y) for d in dots]) +1
            self.width  = max([int(d.x) for d in dots]) +1

        self.sheet = [[None for _ in range(self.width)] for _ in range(self.height)]

        # insert dots
        for d in dots:
            try:
                self.sheet[d.y][d.x] = d
            except (IndexError) as err:
                print(d.x, d.y)

    def __repr__(self) -> str:
        return "".join(
            [
                "".join(['.' if c is None else '#' for c in r]) + "\n"
                for r in p.sheet
            ]
        )

    def fold(self):
        # remove first fold from the list.
        next_fold = self.folds.pop(0)
        print(next_fold)

        if next_fold.axis == 'y':
            for x in range (self.width):
                for y in range(next_fold.position+1, self.height, 1):
                    if self.sheet[y][x] is not None:
                        new_y = next_fold.position - (y - next_fold.position)
                        new_x = x
                        # print(f"{(x,y)} {(new_x, new_y)}") 
                        self.sheet[new_y][new_x] = Dot(new_x, new_y)
                        
            self.height = next_fold.position
        elif next_fold.axis == 'x':
            for x in range(next_fold.position+1, self.width, 1):    
                for y in range (self.height):
                    if self.sheet[y][x] is not None:
                        new_y = y 
                        new_x = next_fold.position - (x - next_fold.position)
                        # print(f"{(x,y)} {(new_x, new_y)}")     
                        self.sheet[y][new_x] = Dot(new_x, new_y)

            self.width = next_fold.position
        
        self.sheet = [sublist[:self.width] for sublist in p.sheet][:self.height]
            

    def count_dots(self) -> int:
        return sum([sum([0 if c is None else 1 for c in r]) for r in p.sheet])

# input data.
def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_13'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_13'
) -> str:
    return os.path.join(data_directory, file_loc)


In [59]:
p = Paper(test_input_location())
print(p.count_dots())
p

18


...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........

In [60]:
p.fold()
assert p.count_dots() == 17
p

Fold(axis='y', position=7)


#.##..#..#.
#...#......
......#...#
#...#......
.#.#..#.###
...........
...........

In [61]:
p.fold()
assert p.count_dots() == 16
p

Fold(axis='x', position=5)


#####
#...#
#...#
#...#
#####
.....
.....

In [62]:
p = Paper(input_location())
print(p.count_dots())
p.fold()
print(p.count_dots())  # after 1 fold

983
Fold(axis='x', position=655)
810


### Part 2
Fold it all and read the 8 digit code that is left.

In [63]:
p = Paper(input_location())
for _ in range(len(p.folds)):
    p.fold()
p

Fold(axis='x', position=655)
Fold(axis='y', position=447)
Fold(axis='x', position=327)
Fold(axis='y', position=223)
Fold(axis='x', position=163)
Fold(axis='y', position=111)
Fold(axis='x', position=81)
Fold(axis='y', position=55)
Fold(axis='x', position=40)
Fold(axis='y', position=27)
Fold(axis='y', position=13)
Fold(axis='y', position=6)


#..#.#....###..#..#.###...##..####.###..
#..#.#....#..#.#..#.#..#.#..#.#....#..#.
####.#....###..#..#.###..#....###..#..#.
#..#.#....#..#.#..#.#..#.#.##.#....###..
#..#.#....#..#.#..#.#..#.#..#.#....#.#..
#..#.####.###...##..###...###.#....#..#.

*If you cannot read that... "HLBUBGFR"*